In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [11]:
df = pd.read_csv('Steel_industry_data.csv')

In [14]:
df.head()

date  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
0  01-01-2018 00:15       3.17                                  2.95   
1  01-01-2018 00:30       4.00                                  4.46   
2  01-01-2018 00:45       3.24                                  3.28   
3  01-01-2018 01:00       3.31                                  3.56   
4  01-01-2018 01:15       3.82                                  4.50   

   Leading_Current_Reactive_Power_kVarh  CO2(tCO2)  \
0                                   0.0        0.0   
1                                   0.0        0.0   
2                                   0.0        0.0   
3                                   0.0        0.0   
4                                   0.0        0.0   

   Lagging_Current_Power_Factor  Leading_Current_Power_Factor   NSM  \
0                         73.21                         100.0   900   
1                         66.77                         100.0  1800   
2                         70.28                         100.0  2700   
3                         68.09                         100.0  3600   
4                         64.72                         100.0  4500   

  WeekStatus Day_of_week   Load_Type  
0    Weekday      Monday  Light_Load  
1    Weekday      Monday  Light_Load  
2    Weekday      Monday  Light_Load  
3    Weekday      Monday  Light_Load  
4    Weekday      Monday  Light_Load

In [15]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df.iloc[:,8] = label_encoder.fit_transform(df.iloc[:,8])
df.iloc[:,9] = label_encoder.fit_transform(df.iloc[:,9])
df.iloc[:,10] = label_encoder.fit_transform(df.iloc[:,10])

In [16]:
### split dataset into independent and dependent features
x = df.iloc[:,[1,3,5,7,8,9]]
y = df.iloc[:,10]

In [17]:
x

Usage_kWh  Leading_Current_Reactive_Power_kVarh  \
0           3.17                                  0.00   
1           4.00                                  0.00   
2           3.24                                  0.00   
3           3.31                                  0.00   
4           3.82                                  0.00   
...          ...                                   ...   
35035       3.85                                  0.00   
35036       3.74                                  0.00   
35037       3.78                                  0.07   
35038       3.78                                  0.11   
35039       3.67                                  0.07   

       Lagging_Current_Power_Factor    NSM  WeekStatus  Day_of_week  
0                             73.21    900           0            1  
1                             66.77   1800           0            1  
2                             70.28   2700           0            1  
3                             68.09   3600           0            1  
4                             64.72   4500           0            1  
...                             ...    ...         ...          ...  
35035                         62.10  82800           0            1  
35036                         70.71  83700           0            1  
35037                         76.62  84600           0            1  
35038                         77.72  85500           0            1  
35039                         77.22      0           0            1  

[35040 rows x 6 columns]

In [23]:
y

0        0
1        0
2        0
3        0
4        0
        ..
35035    0
35036    0
35037    0
35038    0
35039    0
Name: Load_Type, Length: 35040, dtype: int32

In [26]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.30,random_state=0)

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
knn = KNeighborsClassifier(n_neighbors=1)

In [32]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=1)

In [33]:
pred = knn.predict(X_test)

In [34]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

In [35]:
print(confusion_matrix(y_test,pred))

[[5196   48  181]
 [  50 1614  496]
 [ 208  424 2295]]


Row 1 (high load type): The model predicted that 5196 instances were of high load type, and this was correct for 5196 of them. However, it incorrectly predicted 48 instances as medium load type and 181 instances as low load type.

Row 2 (medium load type): The model predicted that 1614 instances were of medium load type, and this was correct for 1614 of them. However, it incorrectly predicted 50 instances as high load type and 496 instances as low load type.

Row 3 (low load type): The model predicted that 2295 instances were of low load type, and this was correct for 2295 of them. However, it incorrectly predicted 208 instances as high load type and 424 instances as medium load type.

In [36]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      5425
           1       0.77      0.75      0.76      2160
           2       0.77      0.78      0.78      2927

    accuracy                           0.87     10512
   macro avg       0.83      0.83      0.83     10512
weighted avg       0.87      0.87      0.87     10512



# Choosing a K value

In [39]:
accuracy_rate = []

for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(knn,x,y,cv=10)
    accuracy_rate.append(score.mean())

In [46]:
error_rate = []
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(knn,x,y,cv=10)
    error_rate.append(1-score.mean())

In [47]:
error_rate = []
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [48]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),accuracy_rate,color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

Text(0, 0.5, 'Error Rate')

In [49]:
# FIRST A QUICK COMPARISON TO OUR ORIGINAL K=1
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=1')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

WITH K=1


[[5196   48  181]
 [  50 1614  496]
 [ 208  424 2295]]


              precision    recall  f1-score   support

           0       0.95      0.96      0.96      5425
           1       0.77      0.75      0.76      2160
           2       0.77      0.78      0.78      2927

    accuracy                           0.87     10512
   macro avg       0.83      0.83      0.83     10512
weighted avg       0.87      0.87      0.87     10512



In [50]:
# NOW WITH K=10
knn = KNeighborsClassifier(n_neighbors=23)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=23')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

WITH K=23


[[5205   31  189]
 [  59 1741  360]
 [ 205  456 2266]]


              precision    recall  f1-score   support

           0       0.95      0.96      0.96      5425
           1       0.78      0.81      0.79      2160
           2       0.80      0.77      0.79      2927

    accuracy                           0.88     10512
   macro avg       0.85      0.85      0.85     10512
weighted avg       0.88      0.88      0.88     10512



In [51]:
# NOW WITH K=10
knn = KNeighborsClassifier(n_neighbors=23)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=23')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

WITH K=23


[[5205   31  189]
 [  59 1741  360]
 [ 205  456 2266]]


              precision    recall  f1-score   support

           0       0.95      0.96      0.96      5425
           1       0.78      0.81      0.79      2160
           2       0.80      0.77      0.79      2927

    accuracy                           0.88     10512
   macro avg       0.85      0.85      0.85     10512
weighted avg       0.88      0.88      0.88     10512

